In [2]:
import pandas as pd
import tomllib

In [3]:
CONFIG_PATH = './data/config.toml'
def load_config(section_name: str) -> dict:
    with open(CONFIG_PATH, 'rb') as config_file:
        return tomllib.load(config_file)[section_name]

In [6]:
df = pd.read_csv('./data/raw_data/pop_census/base-cc-serie-historique-2020.CSV', sep=';', dtype='object')[list(load_config('population').keys())]
a = df.isna().any()
print(*a[a].index, sep='\n')

D82_LOG
D82_LOGVAC
DECE9099
DECE8290
NAIS9099
NAIS8290
D82_RP
D82_RSECOCC


In [12]:
df = pd.read_csv('./data/raw_data/wedding/Dep1.csv', sep=';', dtype='object')

df = df[~((df['REGDEP_MAR'].str.contains('XX')) | (df['REGDEP_MAR'].str.len() < 4))]

df



,TYPMAR3,REGDEP_MAR,GRAGE,NBMARIES
0,FF,1175,14_19,0
1,HF-F,1175,14_19,23
2,HF-H,1175,14_19,1
3,HH,1175,14_19,0
4,FF,1177,14_19,0
...,...,...,...,...
3611,HH,942A,TOTAL,8
3612,FF,942B,TOTAL,16
3613,HF-F,942B,TOTAL,575
3614,HF-H,942B,TOTAL,575


In [14]:
df = pd.read_csv('./data/raw_data/pop_census/base-cc-serie-historique-2020.CSV', sep=';', dtype='object')
df = df[~df['CODGEO'].str.startswith('97')]
df

,CODGEO,P20_POP,P14_POP,P09_POP,D99_POP,D90_POP,D82_POP,D75_POP,D68_POP,SUPERF,...,D75_LOGVAC,D68_LOGVAC,P20_PMEN,P14_PMEN,P09_PMEN,D99_PMEN,D90_NPER_RP,D82_NPER_RP,D75_NPER_RP,D68_NPER_RP
0,01001,806,767,787,728,579,477,368,347,15.95,...,22,25,806,767,787,728,579,477,368,347
1,01002,262,239,207,168,159,136,106,88,9.15,...,23,1,262,239,207,168,159,136,106,88
2,01004,14288,14022,13350,11436,10455,9737,9550,8949,24.6,...,246,155,13787.0089845858,13547.0112272758,12781.9983314098,11015,10092,9333,9164,8582
3,01005,1782,1627,1592,1408,1156,848,756,627,15.92,...,16,15,1782,1627,1592,1406,1155,848,753,627
4,01006,113,109,120,86,76,65,82,108,5.88,...,13,8,113,109,120,86,76,65,82,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34856,95676,490,503,463,426,350,304,253,233,8.25,...,17,24,490,503,463,426,350,304,253,232
34857,95678,857,838,828,775,772,799,590,569,9.82,...,10,5,857,838,811.008379071501,771,772,799,590,569
34858,95680,28636,27571,27013,26145,26110,24808,21876,19119,7.3,...,1067,655,28058.9983683402,27008.9981086448,26357.0089167392,25642,25361,23945,21359,17978
34859,95682,193,177,176,168,161,171,190,223,3.26,...,6,8,193,177,176,168,161,171,190,223
